* Load model trained and saved in previous stage
* * Lora/QLORa model
  

In [1]:
# from google.colab import drive
# drive.mount('/content/drive/')
# # drive.mount('/content/drive/MyDrive/proteins')
# # drive.mount('/content/drive/MyDrive/proteins/New Protein-Virus anom project')
# # !cd /MyDrive/proteins/New\ Protein-Virus\ anom\ project

# # %cd /MyDrive/proteins/New\ Protein-Virus\ anom\ project
# # %cd drive/MyDrive/proteins
# # %cd /New\ Protein-Virus\ anom\ project


# %cd /content/drive/MyDrive/proteins/New\ Protein-Virus\ anom\ project

In [2]:
# # !conda install -c conda-forge google-colab -y

# # !pip install tensorflow  -U -q # ankh

# # !pip install torch  -U -q # fair-esm # seqeval
# !pip install transformers peft accelerate datasets evaluate bitsandbytes wandb -U -q # --user
# # !pip install transformers peft bitsandbytes -U

* Use the unirpot fasta file I downloaded and uploaded to my drive

`/content/drive/MyDrive/Research/biodata/proteins/Transmembrane_human_90.fasta`

* Download fasta from: `https://www.uniprot.org/uniref/?query=uniprot:(keyword%3A%22Transmembrane+%5BKW-0812%5D%22+AND+organism%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22)+identity:0.9`

In [3]:
#### DATA_PATH = "/content/drive/MyDrive/Research/CIDR-Protein Anomalies project/protein_anomalies_data/swp_human_viri_all_embed_esm.parquet" ## ESM1B embedding (max len 1022)
# DATA_PATH = "/content/drive/MyDrive/proteins/New Protein-Virus anom project/hum_vir_swp-globalEmbed-train.csv.gz"## TRAIN
# DATA_PATH = "/content/drive/MyDrive/hum_vir_swp-globalEmbed-train.csv.gz"
# DATA_PATH = "/kaggle/input/humvir-proteins/hum_vir_swp-globalEmbed-train.csv/hum_vir_swp-globalEmbed-train.csv"
DATA_PATH = "hum_vir_swp-globalEmbed-train.csv.gz"

## TEST data:
# TEST_DATA_PATH = "/content/drive/MyDrive/proteins/New Protein-Virus anom project/hum_vir_swp-globalEmbed-test.csv.gz"## TRAIN
# TEST_DATA_PATH = "/content/drive/MyDrive/hum_vir_swp-globalEmbed-test.csv.gz"
# TEST_DATA_PATH = "/kaggle/input/humvir-proteins/hum_vir_swp-globalEmbed-test.csv/hum_vir_swp-globalEmbed-test.csv"
TEST_DATA_PATH = "hum_vir_swp-globalEmbed-test.csv.gz"

# ## metadata for all reviewed/swissprot human + virus proteins
# METADATA_PATH = "/content/drive/MyDrive/Research/CIDR-Protein Anomalies project/protein_anomalies_data/SWP_human_viruses_all.xlsx"

TARGET_COL = "virus" ## use for filtering data into 1 class

MAX_LEN = 612#1024#768#530 # exclude sequences longer than this. (Not merely truncate)

FAST_RUN = False#True
SAVE_MODEL = False#True

# MODEL_DRIVE_SAVE_PATH = "/content/drive/MyDrive/proteins/New Protein-Virus anom project/trained_esm_lora_trainer_model"
# MODEL_DRIVE_SAVE_PATH = "final"
# MODEL_DRIVE_SAVE_PATH = "/kaggle/input/humvir-proteins/qlora/qlora" # saved + reuploadedon kaggle
MODEL_DRIVE_SAVE_PATH =  "./esm_lora_trainer_model_sml" ## LAST used
# MODEL_DRIVE_SAVE_PATH = "./esm150_lora_trainer_model" # nonexist
TRAIN_MODEL = False
LOAD_TRAINED =  True#False# True

In [4]:
import torch
import tensorflow
import tensorflow as tf
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import IsolationForest
# from sklearn.svm import OneClassSVM
from sklearn.preprocessing  import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_predict
# from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
# from sklearn.ensemble import HistGradientBoostingClassifier
from tqdm.autonotebook import tqdm
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, roc_auc_score
from torch import nn
from transformers import TrainingArguments, Trainer, logging
from accelerate import Accelerator
from torch.utils.data.dataloader import DataLoader
## https://huggingface.co/docs/transformers/perf_train_gpu_one

from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AdamWeightDecay
# from tensorflow.keras.optimizers import Adafactor, Adam # more memory effecient than adamWD
# import tensorflow
# from tensorflow.keras.metrics.AUC()
from transformers import AutoTokenizer, EsmForSequenceClassification, Trainer
from transformers import TFAutoModelForSequenceClassification ,TFEsmForSequenceClassification
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer, BitsAndBytesConfig
from peft import AutoPeftModelForSequenceClassification, AutoPeftModel
from transformers import pipeline

## could use transformer pipeline for inference;
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
from peft import PeftModelForSequenceClassification, get_peft_config
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AdamWeightDecay
from tensorflow.keras.optimizers import Adafactor, Adam # more memory effecient than adamWD
import tensorflow
# from tensorflow.keras.metrics.AUC()
# from transformers import AutoTokenizer #DataCollatorForLanguageModeling,
## https://huggingface.co/docs/transformers/model_doc/esm#transformers.EsmForSequenceClassification.forward.example
from transformers import AutoTokenizer, EsmForSequenceClassification, Trainer
from transformers import TFAutoModelForTokenClassification, TFAutoModelForSequenceClassification ,TFEsmForSequenceClassification

from peft import prepare_model_for_kbit_training, get_peft_model
from sklearn.metrics import precision_recall_fscore_support,accuracy_score,roc_auc_score
import evaluate

from datasets import load_metric
import torch._dynamo
torch._dynamo.config.suppress_errors = True

# pipe = pipeline("text-classification", model="facebook/wav2vec2-base-960h", device=0)

2024-03-11 23:06:08.884410: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 23:06:08.902426: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 23:06:08.902448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 23:06:08.903032: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 23:06:08.906540: I tensorflow/core/platform/cpu_feature_guar

In [5]:
# import wandb
# wandb.init(mode="disabled")
# # alt
# wandb.init(project='qlora_humvir')# ; or args = TrainingArguments(report_to="none")

In [6]:
from accelerate import Accelerator
# # Use the accelerator
# ### try disabling? (with qlora)
# # accelerator = Accelerator()# trying this
# # שבבקךקרשאםר צשטנק בשודקד ןדדוקד?
accelerator = Accelerator(mixed_precision="bf16") #bf16") # fp16 # orig used

In [7]:
# model_checkpoint ="facebook/esm2_t6_8M_UR50D"
# model_checkpoint =  "facebook/esm2_t12_35M_UR50D"
# model_checkpoint =  "facebook/esm2_t30_150M_UR50D"
model_checkpoint =  "facebook/esm2_t33_650M_UR50D"

if FAST_RUN:
    # model_checkpoint ="facebook/esm2_t6_8M_UR50D"
  model_checkpoint =  "facebook/esm2_t12_35M_UR50D"


In [8]:
# df = pd.read_parquet(DATA_PATH) # numpy to pandas
df = pd.read_csv(DATA_PATH,usecols=["Sequence","virus","Length","Cluster name"])
df_test = pd.read_csv(TEST_DATA_PATH,usecols=["Sequence","virus","Length",	"Cluster name"])
 ## lengths of all the seqs

if FAST_RUN:
#   # df.loc[df["Length"]>100]
    df = df.sample(frac=0.12,random_state=4)
    df_test = df_test.sample(frac=0.35,random_state=4)
df

,Sequence,virus,Length,Cluster name
0,MADFLKGLPVYNKSNFSRFHADSVCKASNRRPSVYLPTREYPSEQI...,0,102,Cluster: DET1- and DDB1-associated protein 1
1,MPSTLQVLAKKVLALEHKENDHISREYYYHILKCCGLWWHEAPIIL...,1,362,Cluster: Protein MGF 360-19R
2,MASSAELDFNLQALLEQLSQDELSKFKSLIRTISLGKELQTVPQTE...,0,97,Cluster: Pyrin domain-containing protein 2
3,MAAWGKKHAGKDPVRDECEERNRFTETREEDVTDEHGEREPFAETD...,0,186,Cluster: Protein FAM9B
4,MASDSPARSLDEIDLSALRDPAGIFELVELVGNGTYGQVYKGRHVK...,0,1360,Cluster: TRAF2 and NCK-interacting protein kinase
...,...,...,...,...
20335,MDPDKQDALNSIENSIYRTAFKLQSVQTLCQLDLIDSSLIQQVLLR...,0,578,Cluster: Dystrotelin
20336,MLCPWRTANLGLLLILTIFLVAEAEGAAQPNNSLMLQTSKENHALA...,0,348,Cluster: Cell surface glycoprotein CD200 recep...
20337,MCLRFFSPVPGSTSSATNVTMVVSAGPWSSEKAEMNILEINEKLRP...,0,421,Cluster: Putative neuroblastoma breakpoint fam...
20338,MASHAGQQHAPAFGQAARASGPTDGRAASRPSHRQGASEARGDPEL...,1,376,Cluster: Thymidine kinase


In [9]:
ID2LABEL = {
    0: "Human",
    1: "Virus"
}
LABEL2ID = {v: k for k, v in ID2LABEL.items()}

In [10]:
# train_sequences = df["Sequence"].tolist()
# train_labels = df["virus"].tolist()
# train_groups = df["Cluster name"].tolist()

# train_sequences, test_sequences, train_labels, test_labels = train_test_split(sequences, labels, test_size=0.25, shuffle=True)#,stratify=labels)


In [11]:
##https://huggingface.co/docs/peft/main/en/developer_guides/quantization
## lotfQ config - for this, do not initialize as quantized!
# from peft import LoftQConfig, LoraConfig, get_peft_model
# loftq_config = LoftQConfig(loftq_bits=4)

## https://huggingface.co/blog/4bit-transformers-bitsandbytes
compute_dtype = getattr(torch, "bfloat16")

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True, # disable to train ok;
  # load_in_8bit=True, # alt
  bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
    # bnb_4bit_compute_dtype=torch.bfloat16,
    llm_int8_has_fp16_weight =True,
    llm_int8_threshold = 5.1,
   bnb_4bit_compute_dtype=compute_dtype
    ,llm_int8_skip_modules=['classifier',"EsmClassificationHead"] # was enabled?
)

model_max_len = min(1024,MAX_LEN) # 800

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,
                                          padding= True
#                                           padding= True# alt
                                          ,truncation=True,max_length=model_max_len)

num_labels = 2#max(train_labels + test_labels) + 1  # Add 1 since 0 can be a label
print("Num labels:", num_labels)

# ## https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling.ipynb#scrollTo=fc164b49 # uses trainer
# ##

# # model = AutoModelForSequenceClassification. # orig
# # model = EsmForSequenceClassification.
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
#                                                            problem_type="single_label_classification", # was enabled
#                                                            # load_in_4bit=True, # disable to train ok
#                                                             quantization_config=nf4_config,
#                                                           #  load_in_8bit=True,  torch_dtype=torch.float32, # try this - new
#                                                            # device_map= "cuda:0",#"auto",
#                                                            device_map="auto",
#                                                           num_labels=len(ID2LABEL), id2label=ID2LABEL, label2id=LABEL2ID,
#                                                             # trust_remote_code=True
#                                                            # , from_tf=True,
#                                                            # force_download =True,
#                                                           )

# # last_layer_num = model.num_layers ## 33 for esm2_t33_650M_UR50D
# # print(last_layer_num )
# model.train()
# # model.print_trainable_parameters()

Num labels: 2


* `task_type= "SEQ_CLS"` - breaks - RuntimeError: only Tensors of floating point dtype can require gradients

In [12]:
## https://huggingface.co/blog/AmelieSchreiber/esmbind
# https://huggingface.co/docs/peft/task_guides/token-classification-lora
### target modules?? https://discuss.huggingface.co/t/esm-2-qlora-gradient-checkpointing-not-compatible/53505/2
## could also set to all linear?

### https://huggingface.co/docs/peft/task_guides/semantic_segmentation_lora#wrap-the-base-model-as-a-peftmodel-for-lora-training
import bitsandbytes as bnb

# peft_config = LoraConfig(base_model_name_or_path= MODEL_DRIVE_SAVE_PATH #model_checkpoint,
#     ,task_type= TaskType.SEQ_CLS,#"SEQ_CLS",#, ## disabling helps?? (then get "ValueError: Attempting to unscale FP16 gradients.")
#     inference_mode=False, r= 2 if FAST_RUN else 16, #16,
#     lora_alpha=8,
#     # lora_dropout=0.1,
#     use_rslora = True,
#     # bias= "none",#"lora_only",#"none",#"all",
#     # bias="all", #"lora_only",
#     bias= "all",#"lora_only",
#     target_modules=  "all-linear"

# )

# get_peft_model(model, peft_config)

# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

peft_config = LoraConfig(base_model_name_or_path=model_checkpoint, ## MODEL_DRIVE_SAVE_PATH
    task_type= TaskType.SEQ_CLS,
                         r= 16,
                         # lora_alpha=16, use_rslora = True,
                         bias="all",  #"lora_only",#"all", #"lora_only",
           target_modules=  "all-linear" ## causes save score bug; but won't work without this
)

In [13]:
## Warning - longer than allowed length - 1024
def get_train_test_dataseqs(train_sequences,test_sequences,train_labels,test_labels,tokenizer):
    train_tokenized = tokenizer( train_sequences )#,  truncation=True,max_length=model_max_len,padding=True,)
    test_tokenized = tokenizer(test_sequences )#,  truncation=True,max_length=model_max_len,padding=True,)
    train_dataset = Dataset.from_dict(train_tokenized)
    test_dataset = Dataset.from_dict(test_tokenized)

    train_dataset = train_dataset.add_column("labels", train_labels)
    test_dataset = test_dataset.add_column("labels", test_labels)
    return train_dataset,test_dataset

In [14]:
from sklearn.metrics import precision_recall_fscore_support,accuracy_score,roc_auc_score
import evaluate
metric_acc = evaluate.load("accuracy")
metric = evaluate.load("accuracy")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        # 'f1': f1,
        'precision': precision,
        'recall': recall,
        "roc_auc":roc_auc_score(labels,pred.predictions[:,1])
    }

In [15]:
# # ### AttributeError: 'TFEsmForSequenceClassification' object has no attribute 'to'
model_name = model_checkpoint.split("/")[-1]
# print(model_name)

# training_args = TrainingArguments(
#     # f"{model_name}-finetuned-ft-humVir",

#      f"{model_name}-zero_shot_ft",
#     # f"/content/drive/MyDrive/proteins/New Protein-Virus anom project/t{model_name}-finetuned-humVir",
#     per_device_train_batch_size=32,
#     # per_device_eval_batch_size=int(1.5*bch_size),
# #     gradient_accumulation_steps= 2, #4,
#     gradient_checkpointing=True,
#     fp16=True,
#     # bf16=True, # needs ampere, not supported ?
#     # tf32=True,
#         # torch_compile = True,
#     optim = "adamw_8bit", #"paged_adamw_8bit", # adamw_bnb_8bit
#     # optim= "adamw_bnb_8bit", #"paged_adamw_8bit",
#     label_names = ["labels"],
#     learning_rate = 3e-4#2e-4 #5e-3,
#     # lr_scheduler_type="cosine",
#     ,max_grad_norm = 0.95,
#     # weight_decay=0.005,
#     # eval_accumulation_steps = 2#8
#     evaluation_strategy="epoch",
#     save_strategy= "no", #"epoch",#"no",
#     #    load_best_model_at_end=True,
#     # output_dir=".",
#      no_cuda=False,
#      greater_is_better=True,
#      # save_total_limit=1,
#   remove_unused_columns=False,
#     auto_find_batch_size = True, # new , reduces if oom
#     # num_train_epochs=num_epochs,

#     metric_for_best_model= "roc_auc",#"accuracy",
#     group_by_length=True,
# )

# # trainer = Trainer(model=model, args=training_args,
# #                   train_dataset=train_dataset,eval_dataset=test_dataset,tokenizer=tokenizer,
# #                   compute_metrics=compute_metrics,)

In [16]:
# if TRAIN_MODEL:
#     result = trainer.train()

### load model for **comparison**
* , untrained model ?
*  https://huggingface.co/blog/AmelieSchreiber/esmbind

In [17]:
model_checkpoint

'facebook/esm2_t33_650M_UR50D'

In [18]:
### https://huggingface.co/blog/AmelieSchreiber/esmbind
# # ESM2 base model
base_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels,problem_type="single_label_classification")
# base_model.eval()

base_model.gradient_checkpointing_enable()
base_model = prepare_model_for_kbit_training(base_model, use_gradient_checkpointing=True)
base_model = get_peft_model(base_model, peft_config)

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# if FAST_RUN:
#   print("Untrained base model")
#   print(Trainer(model=base_model, args=training_args,
#                   train_dataset=train_dataset,eval_dataset=test_dataset,tokenizer=tokenizer,
#                   compute_metrics=compute_metrics,).evaluate()) # worse than trained model - OK!

In [20]:
MODEL_DRIVE_SAVE_PATH

'./esm_lora_trainer_model_sml'

In [21]:
# trained_model = AutoModelForSequenceClassification.from_pretrained(MODEL_DRIVE_SAVE_PATH,
#                                                               num_labels=num_labels,problem_type="single_label_classification",
#                                                                   id2label=ID2LABEL, label2id=LABEL2ID,) #ORIG, worked

# # trained_model = PeftModel.from_pretrained(trained_model, MODEL_DRIVE_SAVE_PATH) #ORIG, worked
# # alt":
# trained_model = PeftModelForSequenceClassification.from_pretrained(trained_model, MODEL_DRIVE_SAVE_PATH,
#                                                                    config=peft_config) # alt

In [22]:
###
### it says it is not loading saved classifier weights?? but trainer pred gives ok results (better than with random model)?
### load merge_unload model, not just poeft
if LOAD_TRAINED:
    """
    base_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels,problem_type="single_label_classification")
    """
#   ### load pretrained, trained model?

    # trained_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels=num_labels,problem_type="single_label_classification")
    # print("loading PEFT config")

    # trained_model = PeftModelForSequenceClassification.from_pretrained(trained_model, MODEL_DRIVE_SAVE_PATH,
    #                                                                    config=peft_config) # alt

    trained_model = AutoModelForSequenceClassification.from_pretrained(MODEL_DRIVE_SAVE_PATH,
                                                                  num_labels=num_labels,problem_type="single_label_classification",
                                                                      id2label=ID2LABEL, label2id=LABEL2ID,) #ORIG, worked
    merged_model = trained_model ## assumes loaded non peft model

    # # trained_model = PeftModel.from_pretrained(trained_model, MODEL_DRIVE_SAVE_PATH) #ORIG, worked
    # # alt":
    # trained_model = PeftModelForSequenceClassification.from_pretrained(trained_model, MODEL_DRIVE_SAVE_PATH,
    #                                                                    config=peft_config) # alt
    ##
#     trained_model = prepare_model_for_kbit_training(trained_model, use_gradient_checkpointing=True)

else:
    trained_model = trainer.model
# trained_model.eval()

In [23]:
# merged_model = merged_model.merge_and_unload()
merged_model.gradient_checkpointing_enable()
merged_model = prepare_model_for_kbit_training(merged_model, use_gradient_checkpointing=True)
merged_model = get_peft_model(merged_model, peft_config) # added

# res are same!

In [24]:
# merged_model = trained_model.merge_and_unload()
# merged_model

In [25]:
## check if saved loaded ok?
if FAST_RUN:
  print(Trainer(model=trained_model, args=training_args,tokenizer=tokenizer,
                    train_dataset=train_dataset,eval_dataset=test_dataset,
                    compute_metrics=compute_metrics,).evaluate()) ## only ok if better than random init

In [26]:
if FAST_RUN:
  print(trainer.evaluate())

In [27]:
# if FAST_RUN:
#   print("merge and unload peft")

# print(Trainer(model=merged_model, args=training_args,tokenizer=tokenizer,
#                     train_dataset=train_dataset,
#               # eval_dataset=test_dataset,
#                eval_dataset=train_dataset.select(range(700))
#                     ,compute_metrics=compute_metrics,).evaluate())

In [28]:
# print(Trainer(model=base_model, args=training_args,tokenizer=tokenizer,
#                     train_dataset=train_dataset,
#               # eval_dataset=test_dataset,
#                eval_dataset=train_dataset.select(range(700))
#                     ,compute_metrics=compute_metrics,).evaluate())

* Use HF pipeline
* Big bug with pipes - uses cpu? https://discuss.huggingface.co/t/text-classification-pipeline-very-slow-after-adding-padding-and-truncation-for-tokenizer/32173/3
* https://stackoverflow.com/questions/77249578/how-to-create-a-dataset-with-huggingface-from-a-list-of-strings-to-fine-tune-lla"

In [29]:
# https://huggingface.co/docs/transformers/en/main_classes/pipelines#pipeline-batching

# https://stackoverflow.com/questions/68197664/how-to-take-just-the-score-from-huggingface-pipeline-sentiment-analysis

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,
                                          # padding= False#True # orig
                                          padding= True ,truncation=True,max_length=model_max_len)

# pipe = pipeline("text-classification",model=merged_model, tokenizer=tokenizer, truncation=True, top_k=1,device_map="auto") # top_k=None to get preds for each class
## return format is list of dicts, annoying to work with. for k=1 it's just the max class, annoying for a col per class proba score...

* Cao - maybe use only RBD domain ,  (+- mutation site?)


* Could get WT prediction, and get **delta** of mutant's predicted score vs wt - then examine that.



* https://huggingface.co/docs/transformers/model_doc/esm#transformers.EsmForSequenceClassification.forward.example


* Eval related batch size stuff: https://discuss.huggingface.co/t/batch-size-for-trainer-predict/3374/2

In [30]:
# CAO_TEST_DATA_PATH = "/content/drive/MyDrive/proteins/New Protein-Virus anom project/cao_escaper_targets_max.csv"##
# CAO_TEST_DATA_PATH = "/kaggle/input/humvir-proteins/cao_escaper_targets_max.csv"
CAO_TEST_DATA_PATH = "cao_escaper_targets_max.csv"

### all data, more redundnat - same seq repeated, can join with our res to save time.
# CAO_full_TEST_DATA_PATH = "/kaggle/input/humvir-proteins/cao_escaper_targets_all.csv"
CAO_full_TEST_DATA_PATH = "cao_escaper_targets_all.csv.gz"

In [31]:
df_cao = pd.read_csv(CAO_TEST_DATA_PATH)
if FAST_RUN:
  df_cao = df_cao.head(400)

df_cao_full = pd.read_csv(CAO_full_TEST_DATA_PATH)
print(df_cao_full["seq"].nunique())
print(df_cao_full["seq"].str.len())
display(df_cao_full)

3820
0        1273
1        1273
2        1273
3        1273
4        1273
         ... 
34673    1273
34674    1273
34675    1273
34676    1273
34677    1273
Name: seq, Length: 34678, dtype: int64


,site,wildtype,mutation,mut_escape,seq,mut_escape_class
0,331,N,A,0.000000,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,0
1,331,N,C,0.000000,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,0
2,331,N,D,0.000000,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,0
3,331,N,E,0.000000,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,0
4,331,N,F,0.000000,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,0
...,...,...,...,...,...,...
34673,489,Y,R,0.001041,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1
34674,489,Y,S,0.185581,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1
34675,489,Y,T,0.088192,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1
34676,489,Y,V,0.014363,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1


In [32]:
model_checkpoint

'facebook/esm2_t33_650M_UR50D'

In [33]:
from scipy.special import softmax
#OLD
def get_escaper_scores(df:pd.DataFrame,trained_model,base_model,seqColName="seq",truncate_cao=False,
                       binary_target_col = None):
  if truncate_cao:
  ## truncated version +-20:
    cao_sequences = df[seqColName].str[300:550].tolist() # ONLY RBD domain
    max_len = 550-300
  else:
    cao_sequences = df[seqColName].str[0:1024].tolist()
    max_len = df[seqColName].str.len().max()+2

  cao_test_seq = tokenizer(cao_sequences,  truncation=True
                          ,max_length= min(max_len,1024)#1024
                        #  max_length= 700 #600 #model_max_len,
                         ,padding=True#True,
                           # ,padding=True
                           ,return_tensors="pt")

  cao_test_seq = Dataset.from_dict(cao_test_seq)
  # cao_test_seq = accelerator.prepare(cao_test_seq)
    ## get naive models preds - WHY are they similar to trained models ???
  # """
  base_trainer = Trainer(base_model,tokenizer=tokenizer,args=training_args)
  raw_pred, _, _ = base_trainer.predict(cao_test_seq)
  # predictedLabelOnCompanyData = np.argmax(raw_pred, axis=1)
  # df["base_pred_vir"] = raw_pred[:,1]
  df["base_pred_vir"] = torch.nn.functional.softmax(torch.tensor(raw_pred), dim=1)[:,1]
  # """

  ## preds with proper model
  trainer = Trainer(model=trained_model,tokenizer=tokenizer,args=training_args) # added here...
    ### https://discuss.huggingface.co/t/transform-logits-to-probabilities-doesnt-work/14792
  logits, _, _ = trainer.predict(cao_test_seq)

  # print(softmax(logits[:,1]))
  # df["probabilities"] = softmax(logits[:,1]) #(preds_output[0], axis=1)#probabilities

  df["model_pred_vir"] =  torch.nn.functional.softmax(torch.tensor(logits), dim=1)[:,1]
  df["model_pred_hum"] =  torch.nn.functional.softmax(torch.tensor(logits), dim=1)[:,0]
  # df["sum"] = (df["model_pred_hum"]+df["model_pred_vir"]).round(2) # adds up to 1
  df["model_pred_hum_delta"] = df["model_pred_hum"].round(4)

  ## subtract WT pred score - to get deltas. WT is most common (as it reoccurs)
  mode_score = df["model_pred_hum"].round(4).mode()[0]
  df["model_pred_hum_delta"] = df["model_pred_hum_delta"] - mode_score
  print(df.describe().round(3))

  display(df.select_dtypes(["number","bool"]).corr().round(3))

  if binary_target_col is not None:
    print("humVir trained model - human pred delta - ROCAUC:",roc_auc_score(df[binary_target_col], df["model_pred_hum_delta"]).round(4))
    print("humVir trained model ROCAUC:",roc_auc_score(df[binary_target_col], df["model_pred_hum"]).round(4))
    print(classification_report(df[binary_target_col],np.argmax(logits, axis=-1)))
    print("Untrained vir model: ROCAUC",roc_auc_score(df[binary_target_col], df["base_pred_vir"]).round(4))

  return df


## newer, using pipeline (pretrained)
## https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#pipeline-batching
### pipeline approach here is MUCH slower! ?
def pipe_get_escaper_scores(df:pd.DataFrame,trained_model,base_model,seqColName="seq",truncate_cao=False,
                       binary_target_col = None,model_checkpoint='facebook/esm2_t12_35M_UR50D'):
    """
    Much slower than old hacky approach of using a trainer ...
    """
    if truncate_cao:
        ## truncated version +-20:
        cao_sequences = df[seqColName].str[300:550].tolist() # ONLY RBD domain
        max_len = 550-300
    else:
        cao_sequences = df[seqColName].str[0:1024].tolist()
        max_len = df[seqColName].str.len().max()+2

    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,padding= True ,truncation=True,max_length=min(max_len,1024),is_split_into_words=False)

    def do_tokenization(example):
        """
        use like
        dataset = dataset.map(do_tokenization, batched=True)
        """
        return tokenizer(example["text"])

    # cao_sequences = tokenizer(cao_sequences, truncation=True,max_length=model_max_len,is_split_into_words=False) ## ,return_tensors='pt'

    # cao_sequences = Dataset.from_dict(cao_sequences)
    # encoded_dataset = dataset.map(lambda examples: tokenizer(examples['sentence1']),
    #                           batched=True)
    #
    # cao_sequences = Dataset.from_list(tokenizer(cao_sequences,  truncation=True,max_length=model_max_len,padding=False))#.map(tokenizer, batched=True)
    # cao_sequences = Dataset.from_pandas(pd.DataFrame(cao_sequences)).map(tokenizer, batched=True)
    # cao_sequences = Dataset.from_pandas(df[seqColName])

    # cao_test_seq = tokenizer(cao_sequences,  truncation=True
    #                       ,max_length= min(max_len,1024)#1024
    #                     #  max_length= 700 #600 #model_max_len,
    #                      ,padding=False#True,
    #                        # ,padding=True
    #                        ,return_tensors="pt")

    # cao_test_seq = Dataset.from_dict(cao_test_seq)
    # # cao_test_seq = accelerator.prepare(cao_test_seq)
    # ## get naive models preds - WHY are they similar to trained models ???

    def set_vir_label_scores(row):
        """
        Get the virus score per row, (and we can externally set human score from that).
        Assumes model_score col is the k=1 / top score
        """
        if row.model_label.lower() == "virus":
            return row.model_score
        else:
            return 1-row.model_score


    ### naive model - should be random res
    naive_classifier_pipe = pipeline("text-classification",model=base_model, tokenizer=tokenizer,device=0, top_k=1,batch_size=16) # ,device_map="auto"
    # naive_classifier_pipe = pipeline("text-classification",model=base_model, tokenizer=tokenizer, top_k=1,device=0)
    naive_preds = naive_classifier_pipe(cao_sequences)
    naive_preds_labels = [i[0]["label"] for i in naive_preds]
    naive_preds_top_score = [i[0]["score"] for i in naive_preds]
    df["model_label"] = naive_preds_labels
    df["model_label"] = df["model_label"].str.lower()
    df["model_score"] = naive_preds_top_score

    df["base_pred_vir"] = df.apply(set_vir_label_scores, axis=1)
    df.drop(columns=["model_label","model_score"],inplace=True,errors="ignore")

    ## get proepr trained models predictions
    pipe = pipeline("text-classification",model=trained_model, tokenizer=tokenizer, top_k=1,batch_size=16,device=0)  # ,truncation=True
    # pipe = pipeline("text-classification",model=trained_model, tokenizer=tokenizer, top_k=1,device=0)
    ## pipeline (with k=1) returns label, score for top label, need to get per lavbel
    preds = pipe(cao_sequences)
    preds_labels = [i[0]["label"] for i in preds]
    preds_top_score = [i[0]["score"] for i in preds]

    df["model_label"] = preds_labels
    df["model_label"] = df["model_label"].str.lower()
    df["model_score"] = preds_top_score

    df["model_pred_vir"] = df.apply(set_vir_label_scores, axis=1)
    df["model_pred_hum"] = 1-df["model_pred_vir"]
    df["model_pred_hum_delta"] = df["model_pred_hum"].round(4)

    ## subtract WT pred score - to get deltas. WT is most common (as it reoccurs)
    mode_score = df["model_pred_hum"].round(4).mode()[0]
    df["model_pred_hum_delta"] = df["model_pred_hum_delta"] - mode_score
    print(df.describe().round(3))

    display(df.select_dtypes(["number","bool"]).corr().round(3))

    if binary_target_col is not None:
        print("humVir trained model - human pred delta - ROCAUC:",roc_auc_score(df[binary_target_col], df["model_pred_hum_delta"]).round(4))
        print("humVir trained model ROCAUC:",roc_auc_score(df[binary_target_col], df["model_pred_hum"]).round(4))
        # print(classification_report(df[binary_target_col],np.argmax(logits, axis=-1)))
        print(classification_report(df[binary_target_col],df["model_label"]==1))
        print("Untrained vir model: ROCAUC",roc_auc_score(df[binary_target_col], df["base_pred_vir"]).round(4))

    return df

# get_escaper_scores(df_iedb_vir.sample(16).copy(),trained_model=model,base_model=base_model,seqColName="Epitopes - Epitope",truncate_cao=False)

# pipe_get_escaper_scores(df_cao.head(120).copy(),trained_model= merged_model#trainer.model,
#                    ,base_model=base_model,seqColName="seq",truncate_cao=True,
#                        binary_target_col = "mut_escape_class")

In [34]:
df_cao

,site,wildtype,mutation,mut_escape,mut_escape_class,seq
0,331,N,A,0.009428,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
1,331,N,C,0.000000,0,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
2,331,N,D,0.026596,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
3,331,N,E,0.112258,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4,331,N,F,0.012256,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
...,...,...,...,...,...,...
4015,531,T,S,0.010178,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4016,531,T,T,0.000000,0,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4017,531,T,V,0.009521,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4018,531,T,W,0.008336,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


In [35]:
# %%time
# ### 12 sec
# df_cao_preds = get_escaper_scores(df_cao.copy(),trained_model= merged_model#model, #trainer,
#                    ,base_model=base_model,seqColName="seq",truncate_cao=True,
#                        binary_target_col = "mut_escape_class")

# ## fast run. NOTE! trained model res are different than when using pipe!... !
# # humVir trained model - human pred delta - ROCAUC: 0.4858
# # humVir trained model ROCAUC: 0.4858
# # Untrained vir model: ROCAUC 0.4922

In [36]:
# %%time
# ###  slow!
# df_cao_preds = pipe_get_escaper_scores(df_cao.copy(),trained_model= merged_model#trainer.model,
#                    ,base_model=base_model,seqColName="seq",truncate_cao=True,
#                        binary_target_col = "mut_escape_class")

# ### 1 min for all
# ## fast run. NOTE! trained model res are different than when not using pipe!... !
# # humVir trained model - human pred delta - ROCAUC: 0.4798
# # humVir trained model ROCAUC: 0.4798
# # Untrained vir model: ROCAUC 0.4922

In [37]:
# %%time
### works but very slow
# # model_saved(**inputs)
# samples = df_cao.head(70)["seq"].to_list()
# merged_model(**tokenizer(samples,truncation=True,return_tensors='pt'))

In [38]:
# df_cao["model_pred_vir"].describe().round(4)
# df_cao["model_pred_hum"].value_counts() ## most common (271) value - unmutated score/WT

In [39]:
df_cao.select_dtypes(["number","bool"]).corr().round(3)

,site,mut_escape,mut_escape_class
site,1.000,0.073,0.093
mut_escape,0.073,1.000,0.555
mut_escape_class,0.093,0.555,1.000


In [40]:
## merge preds and plot vs escaper score (across many conditions. partially dedupped by cond + identicals)
df_cao_full["seq"].nunique()

3820

### try FT on CAO
* Split by site?

In [41]:
df_cao

,site,wildtype,mutation,mut_escape,mut_escape_class,seq
0,331,N,A,0.009428,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
1,331,N,C,0.000000,0,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
2,331,N,D,0.026596,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
3,331,N,E,0.112258,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4,331,N,F,0.012256,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
...,...,...,...,...,...,...
4015,531,T,S,0.010178,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4016,531,T,T,0.000000,0,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4017,531,T,V,0.009521,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4018,531,T,W,0.008336,1,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


In [42]:
tokenizer_cao = AutoTokenizer.from_pretrained(model_checkpoint,
                                          padding= True
                                          ,truncation=True,max_length= 552#252 ## [300:550]
                                               #df_cao["seq"].str.len().max()
                                          )

In [43]:
train_sequences,test_sequences,train_labels,test_labels = train_test_split(df_cao["seq"].str[0:552],df_cao["mut_escape_class"],
                                                                           stratify=df_cao["mut_escape_class"],test_size=0.25)

train_dataset_immuno_all,test_dataset_immuno = get_train_test_dataseqs(train_sequences.to_list(),test_sequences.to_list(),
                                                                   train_labels,test_labels,tokenizer_cao)

ds_splits = train_dataset_immuno_all.train_test_split(test_size=0.1,seed =1)
# train_dataset_immuno, eval_dataset_immuno = train_dataset_immuno.train_test_split(test_size=0.1)
eval_dataset_immuno = ds_splits["test"]
train_dataset_immuno = ds_splits["train"]

In [44]:
eval_dataset_immuno

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 302
})

In [45]:
train_dataset_immuno

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2713
})

In [46]:
train_dataset_immuno

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2713
})

In [47]:
training_args_cao = TrainingArguments(
    # f"{model_name}-finetuned-ft-humVir",
     f"{model_name}-cao_ft",
    overwrite_output_dir = True,
    # num_train_epochs=1,
    # f"/content/drive/MyDrive/proteins/New Protein-Virus anom project/t{model_name}-finetuned-humVir",
    # per_device_train_batch_size=16,
    # per_device_eval_batch_size=int(1.5*bch_size),
#     gradient_accumulation_steps= 2, #4,
    gradient_checkpointing=True,
    # fp16=True,
    bf16=True, # needs ampere, not supported ?
    tf32=True,
        # torch_compile = True,
    optim =  "paged_adamw_32bit", # "adamw_8bit", #"paged_adamw_8bit", # adamw_bnb_8bit
    # optim= "adamw_bnb_8bit", #"paged_adamw_8bit",
    label_names = ["labels"],
    learning_rate = 4e-4,#2e-4 #5e-3,
    # lr_scheduler_type="cosine",
    # max_grad_norm = 0.95,
    # eval_accumulation_steps = 2#8
    evaluation_strategy="epoch",
    save_strategy= "epoch", #"epoch"#"no",
    load_best_model_at_end=True,
    # weight_decay = 0.001,
    # output_dir=".",
     no_cuda=False,
     greater_is_better=True,
     # save_total_limit=1,
  remove_unused_columns=False,
    # auto_find_batch_size = True, # new , reduces if oom
    # num_train_epochs=num_epochs,
    metric_for_best_model= "eval_roc_auc",#"accuracy",
    # group_by_length=True,
)

trainer_immuno_iedb = Trainer(model=merged_model, args=training_args_cao,
                  train_dataset=train_dataset_immuno,eval_dataset=eval_dataset_immuno,tokenizer=tokenizer_cao,
                  compute_metrics=compute_metrics,)
result_immuno_iedb = trainer_immuno_iedb.train()

wandb: Currently logged in as: ddofer. Use `wandb login --relogin` to force relogin


/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Roc Auc
1,No log,0.690268,0.543046,0.271523,0.500000,0.561528
2,0.718900,0.689738,0.543046,0.271523,0.500000,0.495758
3,0.694400,0.690747,0.543046,0.271523,0.500000,0.474174


/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Checkpoint destination directory esm2_t33_650M_UR50D-cao_ft/checkpoint-340 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./esm_lora_trainer_model_sml - will assume that the vocabulary was not modified.
  warnings.warn(
/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in th

In [48]:
trainer_immuno_iedb.evaluate(test_dataset_immuno)

/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.6919892430305481,
 'eval_accuracy': 0.5283582089552239,
 'eval_precision': 0.26417910447761195,
 'eval_recall': 0.5,
 'eval_roc_auc': 0.503168530040446,
 'eval_runtime': 14.4993,
 'eval_samples_per_second': 69.314,
 'eval_steps_per_second': 8.69,
 'epoch': 3.0}

In [49]:
training_args_cao.output_dir="cao_base"
trainer_immuno_iedb_base = Trainer(model=base_model, args=training_args_cao,
                  train_dataset=train_dataset_immuno,eval_dataset=eval_dataset_immuno,tokenizer=tokenizer_cao,
                  compute_metrics=compute_metrics,)
result_immuno_iedb_base = trainer_immuno_iedb_base.train()

trainer_immuno_iedb_base.evaluate(test_dataset_immuno)

/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Roc Auc
1,No log,0.691471,0.543046,0.271523,0.500000,0.496023
2,0.705500,0.691018,0.543046,0.271523,0.500000,0.484491
3,0.696900,0.690268,0.543046,0.271523,0.500000,0.484226


/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Checkpoint destination directory cao_base/checkpoint-340 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./esm_lora_trainer_model_sml - will assume that the vocabulary was not modified.
  warnings.warn(
/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maint

/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.6915850639343262,
 'eval_accuracy': 0.5283582089552239,
 'eval_precision': 0.26417910447761195,
 'eval_recall': 0.5,
 'eval_roc_auc': 0.5388725992673644,
 'eval_runtime': 29.5186,
 'eval_samples_per_second': 34.046,
 'eval_steps_per_second': 4.268,
 'epoch': 3.0}

Viral escape data:

```
!wget http://cb.csail.mit.edu/cb/viral-mutation/data.tar.gz
!tar xvf data.tar.gz
```
* https://github.com/brianhie/viral-mutation/tree/master/results
* wget from : https://colab.research.google.com/github/vanvalenlab/bebi205/blob/master/bebi205/notebooks/sequences-key.ipynb#scrollTo=VN607f41e_qp
  * relatively large - 3GB zipped!


Example output from them (maybe cov2 seq), with escaper results value:
https://github.com/brianhie/viral-mutation/blob/master/examples/example_results.txt

* another for flu (1, h3)... https://github.com/brianhie/viral-mutation/blob/master/results/flu/semantics/analyze_semantics_flu_h1_bilstm_512.txt

In [50]:
# !wget http://cb.csail.mit.edu/cb/viral-mutation/data.tar.gz
# !tar xvf data.tar.gz
### https://github.com/brianhie/viral-mutation/blob/81c80d41671670eb58cc46e957a1b0c4bf14856a/bin/escape.py#L5
# # pd.read_csv('data/influenza/escape_doud2018/pos_map.csv')

In [51]:
# df_ex = pd.read_csv("https://raw.githubusercontent.com/brianhie/viral-mutation/master/examples/example_results.txt",sep="\t",header=0)

In [52]:
# df_ex = get_escaper_scores(df_ex,trained_model=merged_model,base_model=base_model,seqColName="Sequence",truncate_cao=False)

#### DeepImmuno train data
* https://github.com/frankligy/DeepImmuno/blob/main/reproduce/data/remove0123_sample100.csv
*  this is the train set from there
*  * Deep-learning empowered prediction and generation of immunogenic epitopes for T cell immunity

In [53]:
df_immuno_train = pd.read_csv("DeepImmuno_train_remove0123_sample100.csv").drop_duplicates(subset=["peptide","HLA","immunogenicity"])
df_immuno_train

,peptide,HLA,immunogenicity,test,respond,potential
0,KEHVFFSEY,HLA-B*4402,Negative,4,0,0.347444
1,DEGATLYRF,HLA-B*4402,Negative,4,0,0.346545
2,TLAARIKFL,HLA-A*0201,Negative,4,0,0.346239
3,KETLNEYKQL,HLA-B*4402,Negative,4,0,0.345162
4,STTDAEACY,HLA-A*0101,Negative,4,0,0.343674
...,...,...,...,...,...,...
8966,YYMATLKNV,HLA-A*2402,Positive,50,1,0.370625
8967,ILMNDQEVGV,HLA-A*0201,Positive,89,13,0.356673
8968,ALYEKKLAL,HLA-A*0201,Positive,80,3,0.300920
8969,GIIYIIYKL,HLA-A*0201,Positive,89,4,0.280177


In [54]:
df_immuno_train.nunique()

peptide           8383
HLA                104
immunogenicity       5
test                64
respond             44
potential         8970
dtype: int64

In [55]:
df_immuno_train.immunogenicity.value_counts()

immunogenicity
Negative                 4912
Positive                 3747
Positive-Low              187
Positive-High              87
Positive-Intermediate      38
Name: count, dtype: int64

In [56]:
df_immuno_train["pos_immunogen"] = df_immuno_train["immunogenicity"].str.contains("Positive",case=False,na=False).astype(int)

In [57]:
# df_immuno_train_res = get_escaper_scores(df_immuno_train,trained_model=merged_model,base_model=base_model,seqColName="peptide",truncate_cao=False,binary_target_col="pos_immunogen")

In [58]:
# df_immuno_train_res[['potential',
#        'pos_immunogen', 'base_pred_vir', 'model_pred_vir', 'model_pred_hum',
#        'model_pred_hum_delta']].corr().round(3)#.columns

### Try with fine tuning - deepimmuno
* compare also to baseline model?

In [59]:
tokenizer_immuno = AutoTokenizer.from_pretrained(model_checkpoint,
                                          padding= True
                                          ,truncation=True,max_length=df_immuno_train["peptide"].str.len().max().astype(int))

In [65]:
df_immuno_train["peptide"].str.len().max().astype(int)

10

In [60]:
train_sequences,test_sequences,train_labels,test_labels = train_test_split(df_immuno_train["peptide"],df_immuno_train["pos_immunogen"],
                                                                        stratify=df_immuno_train["pos_immunogen"],test_size=0.35)

train_dataset_immuno_all,test_dataset_immuno = get_train_test_dataseqs(train_sequences.to_list(),test_sequences.to_list(),
                                                                   train_labels,test_labels,tokenizer_immuno)

ds_splits = train_dataset_immuno_all.train_test_split(test_size=0.1,seed =1)
# train_dataset_immuno, eval_dataset_immuno = train_dataset_immuno.train_test_split(test_size=0.1)
eval_dataset_immuno = ds_splits["test"]
train_dataset_immuno = ds_splits["train"]

In [61]:
eval_dataset_immuno

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 584
})

In [62]:
train_dataset_immuno

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5247
})

In [63]:
train_dataset_immuno

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5247
})

In [64]:
training_args_immuno = TrainingArguments(
    # f"{model_name}-finetuned-ft-humVir",
     f"{model_name}-deepImmuno_ft",
     overwrite_output_dir = True,
    # num_train_epochs=4,
    # f"/content/drive/MyDrive/proteins/New Protein-Virus anom project/t{model_name}-finetuned-humVir",
    per_device_train_batch_size=16,
    # per_device_eval_batch_size=int(1.5*bch_size),
#     gradient_accumulation_steps= 2, #4,
    gradient_checkpointing=True,
    # fp16=True,
    bf16=True, # needs ampere, not supported ?
    tf32=True,
        # torch_compile = True,
    # optim = "adamw_8bit", #"paged_adamw_8bit", # adamw_bnb_8bit
    # optim= "adamw_bnb_8bit", #"paged_adamw_8bit",
    label_names = ["labels"],
    learning_rate = 3e-4,#2e-4 #5e-3,
    # lr_scheduler_type="cosine",
    # ,max_grad_norm = 0.95,
    # eval_accumulation_steps = 2#8
    evaluation_strategy="epoch",
    save_strategy= "epoch", #"no", #"epoch"#"no",
    # output_dir=".",
     no_cuda=False,
     greater_is_better=True,
     # save_total_limit=1,
  remove_unused_columns=False,
    # auto_find_batch_size = True, # new , reduces if oom
    # num_train_epochs=num_epochs,
    load_best_model_at_end=True,
    metric_for_best_model= "eval_roc_auc",#"accuracy",
    # group_by_length=True,
)

trainer_immuno = Trainer(model=merged_model, args=training_args_immuno,
                  train_dataset=train_dataset_immuno,eval_dataset=eval_dataset_immuno,tokenizer=tokenizer_immuno,
                  compute_metrics=compute_metrics,)
result_immuno = trainer_immuno.train()

/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Roc Auc
1,No log,0.552209,0.725159,0.722771,0.723302,0.787787


/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./esm_lora_trainer_model_sml - will assume that the vocabulary was not modified.
  warnings.warn(


TypeError: Object of type int64 is not JSON serializable

In [66]:
trainer_immuno.evaluate(test_dataset_immuno)

{'eval_loss': 0.5522086024284363,
 'eval_accuracy': 0.7251592356687898,
 'eval_precision': 0.7227712622666138,
 'eval_recall': 0.7233017657926744,
 'eval_roc_auc': 0.7877871976858386}

In [67]:
training_args_immuno.output_dir="immuno_base"
trainer_immuno_base = Trainer(model=base_model, args=training_args_immuno,
                  train_dataset=train_dataset_immuno,eval_dataset=eval_dataset_immuno,tokenizer=tokenizer_immuno,
                  compute_metrics=compute_metrics,)
result_immuno_base = trainer_immuno_base.train()

/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Roc Auc
1,No log,0.580270,0.714041,0.722424,0.701459,0.789024


/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in ./esm_lora_trainer_model_sml - will assume that the vocabulary was not modified.
  warnings.warn(


TypeError: Object of type int64 is not JSON serializable

In [ ]:
trainer_immuno_base.evaluate(test_dataset_immuno)

### IEDB epitopes
*lots of (short) linear epitopes
* humans, viruses

In [ ]:
df_iedb_human = pd.read_csv("iedb_human_epitopes.csv",compression="zip").drop_duplicates(subset=["Epitopes - Epitope"])
## raw - 1.1M seqs
print(df_iedb_human.nunique())
df_iedb_human

In [ ]:
df_iedb_human[['Epitopes - # References','Epitopes - # Assays']].max(axis=1).describe().round(1)

### filter for higher confidence cases

In [ ]:
df_iedb_human = df_iedb_human.loc[df_iedb_human[['Epitopes - # References','Epitopes - # Assays']].max(axis=1)>1]
print(df_iedb_human.shape[0])
df_iedb_human = df_iedb_human.loc[~df_iedb_human["Epitopes - Epitope"].str.contains("[\+ ]")]
print(df_iedb_human.shape[0])

df_iedb_human = df_iedb_human.loc[df_iedb_human["Epitopes - Epitope"].str.len()>4]

df_iedb_human["labels"] = 0
print(df_iedb_human.shape[0])

In [ ]:
df_iedb_human["Epitopes - Epitope"].str.len().describe().round(1)

In [ ]:
df_iedb_human

In [ ]:
df_iedb_human["Epitopes - Epitope"].str.len().describe().round(0)

In [ ]:
df_iedb_human_preds = get_escaper_scores(df_iedb_human.sample(40_123,).copy(),trained_model=merged_model # model,
                                         ,base_model=base_model,seqColName="Epitopes - Epitope",truncate_cao=False)
df_iedb_human_preds

In [ ]:
df_iedb_human_preds["model_pred_hum"].hist();

In [ ]:
# "E:\Stuff\Research\datasets\New Protein-Virus anom project\iedb_vir_epitopes.csv.zip.csv"
df_iedb_vir = pd.read_csv("iedb_vir_epitopes.csv",compression="zip").drop_duplicates(subset=["Epitopes - Epitope"])
## raw - 138K seqs
print(df_iedb_vir.nunique())
# df_iedb_vir

df_iedb_vir = df_iedb_vir.loc[df_iedb_vir[['Epitopes - # References','Epitopes - # Assays']].max(axis=1)>1]
print(df_iedb_vir.shape[0])
df_iedb_vir = df_iedb_vir.loc[~df_iedb_vir["Epitopes - Epitope"].str.contains("[\+ ]")]
print(df_iedb_vir.shape[0])

df_iedb_vir = df_iedb_vir.loc[df_iedb_vir["Epitopes - Epitope"].str.len()>5]

df_iedb_vir["labels"] = 1
print(df_iedb_vir.shape[0])

In [ ]:
df_iedb_vir["Epitopes - Epitope"].str.len().describe().round(1)

In [ ]:
df_iedb_vir_preds = get_escaper_scores(df_iedb_vir.sample(40_183).copy(),trained_model=merged_model,base_model=base_model,seqColName="Epitopes - Epitope",truncate_cao=False)
df_iedb_vir_preds

In [ ]:
df_iedb_vir_preds["model_pred_hum"].hist();

In [ ]:
df_iedb_vir_preds["label"]="vir"
df_iedb_human_preds["label"]="hum"

In [ ]:
df_iedbs = pd.concat([df_iedb_vir_preds[["label","model_pred_hum"]],df_iedb_human_preds[["label","model_pred_hum"]]])
df_iedbs

In [ ]:
### FIG 3

In [ ]:
import seaborn as sns
# https://stackoverflow.com/questions/28293028/plotting-grouped-data-in-same-plot-using-pandas
# df_iedbs.groupby(["label"]).plot(kind='kde', ax=plt.gca())

# sns.kdeplot(data=df_iedbs, x="model_pred_hum", hue="label")#, bw_adjust=1, cut=1)
df2 = df_iedbs.copy()
df2.rename(columns={"model_pred_hum":"Predicted Humanness score"},inplace=True)
df2["label"] = df2["label"].str.replace("vir","Virus").str.replace("hum","Human")
sns.kdeplot(data=df2, x="Predicted Humanness score", hue="label")

In [ ]:
df_iedbs.groupby("label").describe().round(2)

### Try FT on iedb peptides

In [ ]:
df_iedbs

### Try with fine tuning - deepimmuno
* compare also to baseline model?

In [ ]:
df_iedb_sample = pd.concat([df_iedb_vir.sample(35_000),df_iedb_human.sample(35_000)]).filter(["Epitopes - Epitope","labels"])

df_iedb_sample.rename(columns={"Epitopes - Epitope":"peptide"},inplace=True)
df_iedb_sample

In [ ]:
df_iedb_sample.peptide.str.len().describe()

In [ ]:
tokenizer_iedb = AutoTokenizer.from_pretrained(model_checkpoint,
                                          padding= True
                                          ,truncation=True,max_length=df_iedb_sample["peptide"].str.len().max().astype(int))

In [ ]:
train_sequences,test_sequences,train_labels,test_labels = train_test_split(df_iedb_sample["peptide"],df_iedb_sample["labels"],
                                                                           stratify=df_iedb_sample["labels"],test_size = 0.35)

train_dataset_immuno_all,test_dataset_immuno = get_train_test_dataseqs(train_sequences.to_list(),test_sequences.to_list(),
                                                                   train_labels,test_labels,tokenizer_iedb)

ds_splits = train_dataset_immuno_all.train_test_split(test_size=0.1,seed =1)
# train_dataset_immuno, eval_dataset_immuno = train_dataset_immuno.train_test_split(test_size=0.1)
eval_dataset_immuno = ds_splits["test"]
train_dataset_immuno = ds_splits["train"]

In [ ]:
eval_dataset_immuno

In [ ]:
train_dataset_immuno

In [ ]:
train_dataset_immuno

In [ ]:
training_args_immuno = TrainingArguments(
    # f"{model_name}-finetuned-ft-humVir",
     f"{model_name}-iedbImmuno_ft",
     overwrite_output_dir = True,
    # num_train_epochs=1,
    # f"/content/drive/MyDrive/proteins/New Protein-Virus anom project/t{model_name}-finetuned-humVir",
    per_device_train_batch_size=16,
    # per_device_eval_batch_size=int(1.5*bch_size),
#     gradient_accumulation_steps= 2, #4,
    gradient_checkpointing=True,
    # fp16=True,
    bf16=True, # needs ampere, not supported ?
    tf32=True,
        torch_compile = True,
    # optim = "adamw_8bit", #"paged_adamw_8bit", # adamw_bnb_8bit
    # optim= "adamw_bnb_8bit", #"paged_adamw_8bit",
    label_names = ["labels"],
    learning_rate = 3e-4,#2e-4 #5e-3,
    # lr_scheduler_type="cosine",
    # max_grad_norm = 0.95,
    # eval_accumulation_steps = 2#8
    evaluation_strategy="epoch",
    save_strategy="epoch", #"epoch"#"no",
        load_best_model_at_end=True,
    # weight_decay = 0.001,
    # output_dir=".",
     no_cuda=False,
     greater_is_better=True,
     # save_total_limit=1,
  remove_unused_columns=False,
    # auto_find_batch_size = True, # new , reduces if oom
    # num_train_epochs=num_epochs,

    metric_for_best_model= "eval_roc_auc",#"accuracy",
    # group_by_length=True,
)

trainer_immuno_iedb = Trainer(model=merged_model, args=training_args_immuno,
                  train_dataset=train_dataset_immuno,eval_dataset=eval_dataset_immuno,tokenizer=tokenizer_iedb,
                  compute_metrics=compute_metrics,)
result_immuno_iedb = trainer_immuno_iedb.train()

In [ ]:
trainer_immuno_iedb.evaluate(test_dataset_immuno)

In [ ]:
training_args_immuno.output_dir="iedb_base"
trainer_immuno_iedb_base = Trainer(model=base_model, args=training_args_immuno,
                  train_dataset=train_dataset_immuno,eval_dataset=eval_dataset_immuno,tokenizer=tokenizer_iedb,
                  compute_metrics=compute_metrics,)
result_immuno_iedb_base = trainer_immuno_iedb_base.train()

In [ ]:
trainer_immuno_iedb_base.evaluate(test_dataset_immuno)